# Data Analysis and Visualisation &mdash; Lab 04

## Problem 3

In [2]:
import re
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc
from dash.dependencies import Input, Output

### Loading data

In [ ]:
df = pd.read_csv("../data/laptop-prices/Laptop+Prices.csv")

### Plotly Dash

**Explanation:**

This Dash app provides an interactive dashboard for exploring laptop prices. It contains two main sections:

* Prices by Brands: Users can filter laptops by brand using a dropdown, and the bar chart updates dynamically to show min, max, and average prices for the selected brands.
* Prices by RAM: A range slider allows users to select RAM capacities, and the horizontal boxplot updates to show the price distribution for laptops within that RAM range.

Callbacks link the input controls to the figures, enabling real-time filtering and visualization updates without reloading the page.

#### Adding extra info

In [114]:
os_patterns = {
    "Windows": r"Windows",
    "macOS": r"macOS|OS X",
    "Linux": r"Linux",
    "Chrome OS": f"Chrome OS",
}

def extract_processor_series(model):
    match = re.search(r"(Core i[357]|Xeon|Pentinum|Celeron|Ryzen)", model, re.IGNORECASE)
    return match.group(1) if match else "Other"

def extract_graphics_card_series(model):
    match = re.search(r"(GeForce (GTX|RTX)|Radeon|(UHD|HD|Iris.*) Graphics)", model, re.IGNORECASE)
    return match.group(1) if match else "Other"

def categorize_os(os):
    for group, pattern in os_patterns.items():
        if re.search(pattern, os, re.IGNORECASE):
            return group
    return "Other"

df["CPU_series"] = df["CPU_model"].apply(extract_processor_series)
df["CPU_type"] = (
    df["CPU_company"].fillna("") + " " + df["CPU_series"].fillna("")
).str.strip()

df["GPU_series"] = df["GPU_model"].apply(extract_graphics_card_series)
df["GPU_type"] = (
    df["GPU_company"].fillna("") + " " + df["GPU_series"].fillna("")
).str.strip()

df["OS_group"] = df["OS"].apply(categorize_os)

#### Plot functions

In [44]:
def get_brand_prices_chart(df):
    return px.bar(
        (df
            .groupby("Company")["Price_euros"]
            .agg(["min", "max", "mean"])
            .reset_index()
            .sort_values("mean")
        ),
        x="Company", y=["min", "max", "mean"],
        barmode="group",
    )

In [85]:
def get_prices_ram_chart(df):
    ram_counts = df["Ram"].value_counts()
    
    new_df = df[df["Ram"].isin(ram_counts[ram_counts > 1].index)]
    new_df = new_df.assign(Ram_str=new_df["Ram"].astype(str) + "GB")
    
    fig = px.box(
        new_df,
        x="Price_euros", y="Ram_str", color="Ram_str",
        orientation="h", points=False,
    )

    fig.update_yaxes(
        categoryorder="array",
        categoryarray=[f"{ram}GB" for ram in sorted(new_df["Ram"].unique(), reverse=True)],
    )

    return fig

#### Main app

In [152]:
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Laptop Dashboard", style={"textAlign": "center"}),
    
    # ====================
    # Prices by Brands
    # ====================
    html.Div([
        html.H2("Prices by Brands"),
        html.Div([
            dcc.Dropdown(
                id="brand_prices_filters_brand",
                options=[
                    {"label": brand, "value": brand}
                    for brand in df["Company"].unique()
                ],
                multi=True,
                placeholder="Select brands...",
            ),
        ], style={
            "display": "grid",
            "grid-template-columns": "1fr 1fr",
        }),
        dcc.Graph(
            id="brand_prices_chart",
            figure=get_brand_prices_chart(df),
        ),
    ]),

    # ====================
    # Prices by RAM
    # ====================
    html.Div([
        html.H2("Prices by RAM"),
        html.Div([
            dcc.RangeSlider(
                id="prices_ram_filters_ram",
                min=2, max=32, step=2, value=[2, 32],
                marks={
                    i: f"{i}GB" if i in df["Ram"].unique() else ""
                    for i in range(2, 33, 2)
                },
            ),
        ], style={
            "display": "grid",
            "grid-template-columns": "1fr",
        }),
        dcc.Graph(
            id="prices_ram_chart",
            figure=get_prices_ram_chart(df),
        ),
    ]),
])

@app.callback(
    Output("brand_prices_chart", "figure"),
    Input("brand_prices_filters_brand", "value"),
)
def update_brand_prices_chart(selected_brands):
    return get_brand_prices_chart(
        df[df["Company"].isin(selected_brands)]
        if selected_brands else df
    )

@app.callback(
    Output("prices_ram_chart", "figure"),
    Input("prices_ram_filters_ram", "value"),
)
def update_prices_ram_chart(ram_range):
    return get_prices_ram_chart(df[
        (df["Ram"] >= ram_range[0]) &
        (df["Ram"] <= ram_range[1])
    ])

app.run(jupyter_mode="inline", debug=True)